In [2]:
import requests
import base64
import datetime
import os

# GitHub API URL for the repository's commits
repo_commits_url = 'https://api.github.com/repos/SMOKAL/AutoGit/commits'
# The path to the folder in the repository
folder_path = 'Rules/'
#Get the token from env variable
token=os.getenv('TOKENGIT')
# Your GitHub personal access token (optional, but may be required for private repositories or higher rate limits)
headers = {
    'Authorization': f'token {token}',
    'Accept': 'application/vnd.github.v3+json'
}
# Function to get the most recent commit affecting the folder
def get_most_recent_commit(repo_url, folder):
    # Make a request to get the commits
    params = {'path': folder}
    response = requests.get(repo_url, headers=headers, params=params)
    response.raise_for_status()  # This will raise an exception for HTTP error codes
    # Get the list of commits
    commits = response.json()
    # Check if there are any commits
    if not commits:
        print("No commits found for the specified folder.")
        return None
    # Get the most recent commit
    recent_commit = commits[0]
    return recent_commit
# Function to get the list of files changed in a commit
def get_commit_files(commit_sha):
    commit_url = f"{repo_commits_url}/{commit_sha}"
    response = requests.get(commit_url, headers=headers)
    response.raise_for_status()  # This will raise an exception for HTTP error codes
    commit_data = response.json()
    return commit_data['files']


#Search Operation to get the files in all the folder which have Start and End patter
def fetch_files_with_pattern(repo_owner, repo_name, start_pattern, end_pattern):
    headers = {'Accept': 'application/vnd.github.v3+json'}
    matching_files = []

    def search_folder(folder_path):
        base_url = f"https://api.github.com/repos/{repo_owner}/{repo_name}/contents/{folder_path}"
        response = requests.get(base_url, headers=headers)

        if response.status_code != 200:
            raise Exception(f"Failed to fetch repository contents for '{folder_path}': {response.status_code}")

        repo_contents = response.json()

        for file_info in repo_contents:
            if file_info['type'] == 'file':
                file_url = file_info['download_url']
                file_response = requests.get(file_url)

                if file_response.status_code != 200:
                    raise Exception(f"Failed to fetch file contents: {file_response.status_code}")

                file_content = file_response.text
                if start_pattern in file_content and end_pattern in file_content:
                    matching_files.append(file_info['path'])  # Use 'path' to include the folder path
            elif file_info['type'] == 'dir':
                search_folder(file_info['path'])  # Recursively search this directory

    # Start the search from the root of the repository
    search_folder('')

    return matching_files
    
#This will generate the commit message after the content get overwrite between the start and end pattern in the matching files.    
def generate_commit_message(BusinessLogic, content_type):
    current_time = datetime.datetime.now()
    message = f"Update::{BusinessLogic}::{content_type} on {current_time.strftime('%Y-%m-%d %H:%M:%S')}"
    return message

#This will write the content between the start and end pattern in the matching files.
def overwrite_content_in_files(repo_owner, repo_name, matching_files, start_pattern, end_pattern, new_content, token):
    headers = {'Accept': 'application/vnd.github.v3+json','Authorization': f'token {token}'}
    for file_path in matching_files:
        # Get the file content and SHA
        content_url = f"https://api.github.com/repos/{repo_owner}/{repo_name}/contents/{file_path}"
        content_response = requests.get(content_url, headers=headers)
        
        if content_response.status_code != 200:
            raise Exception(f"Failed to fetch file content for '{file_path}': {content_response.status_code}")
                    
        content_info = content_response.json()
        file_content = base64.b64decode(content_info['content']).decode('utf-8')
        sha = content_info['sha']

        # Find and replace the content between the start and end patterns
        start_index = file_content.find(start_pattern) + len(start_pattern)
        end_index = file_content.find(end_pattern)
        updated_content = file_content[:start_index] + new_content + file_content[end_index:]

        # Encode the updated content to base64
        encoded_content = base64.b64encode(updated_content.encode('utf-8')).decode('utf-8')

        # Prepare the update payload
        BusinessLogic=filename_without_extension
        dynamic_message = generate_commit_message(BusinessLogic,"Sqlcode")
        update_payload = {'message': dynamic_message ,'content': encoded_content,'sha': sha,'branch': 'main',  
                          # Specify the branch if not 'main'
                         }
        
        # Send the update request
        update_response = requests.put(content_url, headers=headers, json=update_payload)
        if update_response.status_code not in (200, 201):
            raise Exception(f"Failed to update file '{file_path}': {update_response.status_code}")


# Fetch the most recent commit for the folder
recent_commit = get_most_recent_commit(repo_commits_url, folder_path)

# Fetch and print the filenames of the most recent commit
if recent_commit:
    commit_sha = recent_commit['sha']
    print(f"Most recent commit SHA: {commit_sha}")
    print("Files changed in the most recent commit:")
    commit_files = get_commit_files(commit_sha)
    for file in commit_files:
        file_path = file['filename']
        print({file_path})
        # Extract the filename with extension
        filename_with_extension = os.path.basename(file_path)

        # Split the extension and get just the filename
        filename_without_extension, _ = os.path.splitext(filename_with_extension)
        print(filename_without_extension)
        url = f"https://raw.githubusercontent.com/SMOKAL/AutoGit/{commit_sha}/{file_path}"
        response1 = requests.get(url)
        if response1.status_code == 200:
            content=response1.text
        else:
            print(f'There was  different operation other than Update: {response1.status_code} {response1.reason}')
            content='None';
        print('\n')
        print(f'and the content of the file is: {'\n'} {content}')

#The Information for usage
repo_owner = 'SMOKAL'
repo_name = 'AutoGit'
#start_pattern = f"/** Rule Start :{filename_without_extension}     **/"
#end_pattern = f"/** Rule End :{filename_without_extension}     **/"
start_pattern = f"#RuleStart:{filename_without_extension}#"
end_pattern = f"#RuleEnd:{filename_without_extension}#"
new_content = content
  # Replace with your GitHub personal access token


try:
    matching_files = fetch_files_with_pattern(repo_owner, repo_name, start_pattern, end_pattern)
    print("Matching filenames:")
    for filename in matching_files:
        print(filename)
    overwrite_content_in_files(repo_owner, repo_name, matching_files, start_pattern, end_pattern, new_content, token)
    print("Content between patterns has been updated.")
except Exception as e:
    print(f"An error occurred: {e}")

Most recent commit SHA: 1e8581326142b45f2d61a1eacea4db8208290d7c
Files changed in the most recent commit:
{'Rules/Rule6ValidationProof.sql'}
Rule6ValidationProof


and the content of the file is: 
    
cte1 as
 (Select v.validationid,b.time from table1
  where v.status='Married'),
Rule6ValidationProof as 
  (Select s.id,s.name,c.validationid from soap s join
   cte1 c)Changedoinginthecode123

Matching filenames:
Report/Report3.sql
ReportDummy/Report3Dummy.sql
Content between patterns has been updated.
